In [143]:
seq_len = 24 * 4 * 4

border1s = [0, 12 * 30 * 24 + 4 * 30 * 24 - seq_len, 12 * 30 * 24 + 4 * 30 * 24 - seq_len]
border2s = [12 * 30 * 24 + 4 * 30 * 24, 12 * 30 * 24 + 8 * 30 * 24, 12 * 30 * 24 + 8 * 30 * 24]

flag = 'val'
type_map = {'train': 0, 'val': 1, 'test': 2}
set_type = type_map[flag]

border1 = border1s[set_type]
border2 = border2s[set_type]

In [146]:
231// 10000

0

In [145]:
print(border1,border2,border2-border1)

11136 14400 3264


In [29]:
import torch
from data_provider.data_loader import Dataset_ETT_hour
def calcute_lags(x_enc):
    q_fft = torch.fft.rfft(x_enc.permute(0, 2, 1).contiguous(), dim=-1)
    k_fft = torch.fft.rfft(x_enc.permute(0, 2, 1).contiguous(), dim=-1)
    res = q_fft * torch.conj(k_fft)
    corr = torch.fft.irfft(res, dim=-1)
    mean_value = torch.mean(corr, dim=1)
    _, lags = torch.topk(mean_value, 1, dim=-1)
    return lags

In [147]:
from data_provider.data_loader import Dataset_ETT_hour, Dataset_ETT_minute, Dataset_Custom, Dataset_M4
from torch.utils.data import DataLoader

data_dict = {
    'ETTh1': Dataset_ETT_hour,
    'ETTh2': Dataset_ETT_hour,
    'ETTm1': Dataset_ETT_minute,
    'ETTm2': Dataset_ETT_minute,
    'ECL': Dataset_Custom,
    'Traffic': Dataset_Custom,
    'Weather': Dataset_Custom,
    'm4': Dataset_M4,
}

def data_provider(flag):
    data = 'ETTh1'
    embed = 'timeF'
    percent = 100
    batch_size = 32
    freq = 'h'
    root_path = './'
    data_path = 'ETTh1.csv'
    seq_len = 512
    label_len = 48
    pred_len = 96
    features = 'S'
    target = 'OT'
    seasonal_patterns = 'Monthly'
    num_workers = 1
    Data = data_dict[data]
    timeenc = 0 #if embed != 'timeF' else 1
    percent = 100

    if flag == 'test':
        shuffle_flag = False
        drop_last = True
        batch_size = batch_size
        freq = freq
    else:
        shuffle_flag = True
        drop_last = True
        batch_size = batch_size
        freq = freq
    data_set = Data(
        root_path=root_path,
        data_path=data_path,
        flag=flag,
        size=[seq_len, label_len, pred_len],
        features=features,
        target=target,
        timeenc=timeenc,
        freq=freq,
        percent=percent,
        seasonal_patterns=seasonal_patterns,
        scale = False
    )
    data_loader = DataLoader(
        data_set,
        batch_size=batch_size,
        shuffle=shuffle_flag,
        num_workers=num_workers,
        drop_last=drop_last)
    return data_set, data_loader

In [148]:
dataset = Dataset_ETT_hour(root_path='./',scale=False)

In [149]:
train_data, train_loader = data_provider(flag = 'train')

In [150]:
print(len(train_data[0][0]))
print(type(train_data[0]))
print(train_data[0][0])

512
<class 'tuple'>
[[30.53100014]
 [27.78700066]
 [27.78700066]
 [25.04400063]
 [21.94799995]
 [21.17399979]
 [22.79199982]
 [23.1439991 ]
 [21.66699982]
 [17.44599915]
 [19.97900009]
 [20.11899948]
 [19.20499992]
 [18.5720005 ]
 [19.55599976]
 [17.30500031]
 [19.48600006]
 [19.13400078]
 [20.68199921]
 [18.71199989]
 [17.86800003]
 [18.00900078]
 [18.00900078]
 [19.76799965]
 [21.10400009]
 [19.6970005 ]
 [20.04899979]
 [20.75200081]
 [21.38500023]
 [22.22999954]
 [20.26000023]
 [21.10400009]
 [20.61199951]
 [18.36100006]
 [20.96299934]
 [19.41600037]
 [20.82299995]
 [20.19000053]
 [21.31500053]
 [22.0189991 ]
 [20.68199921]
 [25.4659996 ]
 [25.88800049]
 [27.85700035]
 [27.29500008]
 [22.22999954]
 [21.94799995]
 [27.29500008]
 [29.33499908]
 [26.02799988]
 [24.34000015]
 [26.45000076]
 [25.95800018]
 [24.05900002]
 [25.32500076]
 [23.63699913]
 [26.37999916]
 [27.36499977]
 [28.06800079]
 [29.47500038]
 [26.80200005]
 [29.96800041]
 [30.38999939]
 [31.16399956]
 [29.75699997]
 [32.

In [151]:
from tqdm import tqdm
for (batch_x, batch_y, batch_x_mark, batch_y_mark) in train_loader:
    batch_x_1 = batch_x
    batch_y_1 = batch_y
    batch_x_mark_1 = batch_x_mark
    batch_y_mark_1 = batch_y_mark
    break

In [152]:
print(batch_x[0][0])
print(batch_x[0][-1])

tensor([25.8880], dtype=torch.float64)
tensor([22.1590], dtype=torch.float64)


In [153]:
batch_y_1[0][:10]

tensor([[20.2600],
        [20.6120],
        [19.9790],
        [20.0490],
        [19.7680],
        [20.4010],
        [20.6820],
        [21.4560],
        [22.0890],
        [22.5810]], dtype=torch.float64)

In [154]:
print(batch_x_mark_1[0][0])
print(batch_x_mark_1[0][-1])

tensor([ 9, 13,  1, 19])
tensor([10,  5,  2,  2])


In [155]:
print(batch_y_mark_1[0][0])
print(batch_y_mark_1[0][-1])

tensor([10,  3,  0,  3])
tensor([10,  9,  6,  2])


In [156]:
a = [1,2,3,4,5]
print(a[-2:])

[4, 5]


In [162]:
a = torch.tensor([[1, 2, 3],
                  [1, 2, 3]])
b = a.repeat(2,2)
print(b.shape)
print(b)

torch.Size([4, 6])
tensor([[1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3]])
